# Improving Employee Retention By Predicting Employee Attrition Using Machine Learning

**Project Overview**:<br>
In this Mini Project, I have a role as a member of the Data Scientist team at a technology start-up company. The company is currently experiencing big problems; many employees have submitted their resignations, but the company has not yet decided on this matter. I will help the company to explain the current condition of its employees, as well as explore the problems within the company that cause employees to resign so that they can reduce the rate of employee resignations and outline a strategy that can increase employee retention. I will present descriptive findings from data using data visualization and data storytelling. I can get inferential findings using statistical analysis, or machine learning approaches with frameworks such as Interpretable & Explainable AI to convert model findings into a story.<br><br>

**Project Goals**:<br>
1. *Reduce Employee Resignation/Attrition Rate*.<br>This is the ultimate goal this project aims to achieve. By understanding the reasons behind employee resignations, the company can take stepsto make the workplace more attractive and retaint the employees.
2. *Improve employee retention*.<br>Retaining existing employee is generally cheaper and more efficient than hiring and training new ones. This project aims to identify strategies that will encourage employees to stay with the company.<br>

**Project Objective**:<br>
1. Understand the current state of employee sentiment.
2. Identify the root causes of employee resignations.
3. Develop data-driven recommendations to improve employee retention.
4. Communicate insights effectively.
5. Develop a predictive model to predict employee attrition.<br>

**Project Business Metrics**:<br>
1. Attrition Rate.
2. Time to fill open positions.